In [1]:
#!pip install tensorflow
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import os
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
credits= pd.read_csv("/content/drive/MyDrive/dataIA/movies/credits.csv")
keywords= pd.read_csv("/content/drive/MyDrive/dataIA/movies/keywords.csv")
movies = pd.read_csv("/content/drive/MyDrive/dataIA/movies/movies_metadata.csv")
#imdb_movies = pd.read_csv("/content/drive/MyDrive/dataIA/movies/IMDB Dataset.csv")

<ipython-input-2-6b784f169dc2>:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("/content/drive/MyDrive/dataIA/movies/movies_metadata.csv")


In [3]:
# Obtener los nombres de las columnas y verificar si contienen valores nulos
# Obtener los nombres de las columnas y verificar si contienen valores nulos
datasets = {"credits": credits, "keywords": keywords, "movies": movies, "imdb_movies": imdb_movies}
for name, dataset in datasets.items():
    print("Nombre del DataFrame:", name)
    print("Nombres de las columnas:", dataset.columns.tolist())
    print("Contiene nulos:", dataset.isnull().any().any())
    print("\n")

Nombre del DataFrame: credits
Nombres de las columnas: ['cast', 'crew', 'id']
Contiene nulos: False


Nombre del DataFrame: keywords
Nombres de las columnas: ['id', 'keywords']
Contiene nulos: False


Nombre del DataFrame: movies
Nombres de las columnas: ['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']
Contiene nulos: True


Nombre del DataFrame: imdb_movies
Nombres de las columnas: ['review', 'sentiment']
Contiene nulos: False




In [4]:
# Nos quedamos con todas las movies que sean status=Released
# campos interesantes: adults, genres, id AKA id_movies, original_title, overview,
# release_date, spoken_languages, tagline (opinion), vote_average
movies_new = movies[movies['status'] == 'Released'][['adult', 'genres', 'id', 'imdb_id',
                                        'overview', 'release_date', 'runtime', 'spoken_languages', 'tagline', 'title', 'vote_average']]
print(movies_new.head())

   adult                                             genres     id    imdb_id  \
0  False  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...    862  tt0114709   
1  False  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   8844  tt0113497   
2  False  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...  15602  tt0113228   
3  False  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  31357  tt0114885   
4  False                     [{'id': 35, 'name': 'Comedy'}]  11862  tt0113041   

                                            overview release_date  runtime  \
0  Led by Woody, Andy's toys live happily in his ...   1995-10-30     81.0   
1  When siblings Judy and Peter discover an encha...   1995-12-15    104.0   
2  A family wedding reignites the ancient feud be...   1995-12-22    101.0   
3  Cheated on, mistreated and stepped on, the wom...   1995-12-22    127.0   
4  Just when George Banks has recovered from his ...   1995-02-10    106.0   

                                    spoken_l

Para el preprocesamiento se ha pensado en rellenar todas las filas que no contengan idioma asignado a "English" para tenerlas disponibles.

In [5]:
# Para la parte de idiomas disponibles agrego [{'iso_639_1': 'en', 'name': 'English'}] a las filas vacías
movies_new['spoken_languages'] = movies_new['spoken_languages'].apply(lambda x: [{'iso_639_1': 'en', 'name': 'English'}] if pd.isna(x) or x == '[]' else eval(x))

In [6]:
spoken_languages_column = movies_new['spoken_languages']
all_language_names = []
for languages_list in spoken_languages_column:
    language_names = []
    for language_dict in languages_list:
        language_names.append(language_dict['name'])
    all_language_names.append(language_names)
#print(all_language_names)
movies_new['languages_availables'] = all_language_names
movies_new.head()

,adult,genres,id,imdb_id,overview,release_date,runtime,spoken_languages,tagline,title,vote_average,languages_availables
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,[English]
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6.9,"[English, Français]"
2,False,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,[English]
3,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,[English]
4,False,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,[English]


Para el preprocesamiento de la columna "genres" se ha pensado hacer lo mismo que con los idiomas para identificar los géneros que califican la película

In [7]:
import ast

genres_column = movies_new['genres']

# Convertir las cadenas de texto que representan listas de diccionarios en listas de diccionarios reales
genres_lists = genres_column.apply(ast.literal_eval)

all_genre_names = []
for genres_list in genres_lists:
    genre_names = []
    for genre_dict in genres_list:
        genre_names.append(genre_dict['name'])
    all_genre_names.append(genre_names)

# Mostrar los nombres de género obtenidos
#print(all_genre_names)
movies_new['genres_associated'] = all_genre_names
movies_new.head()

,adult,genres,id,imdb_id,overview,release_date,runtime,spoken_languages,tagline,title,vote_average,languages_availables,genres_associated
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,[English],"[Animation, Comedy, Family]"
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6.9,"[English, Français]","[Adventure, Fantasy, Family]"
2,False,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,[English],"[Romance, Comedy]"
3,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,[English],"[Comedy, Drama, Romance]"
4,False,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,[English],[Comedy]


In [8]:
# Se borran las columnas que se han procesado
movies_new.drop(columns=['genres', 'spoken_languages'], inplace=True)
print(movies_new.head())

   adult     id    imdb_id                                           overview  \
0  False    862  tt0114709  Led by Woody, Andy's toys live happily in his ...   
1  False   8844  tt0113497  When siblings Judy and Peter discover an encha...   
2  False  15602  tt0113228  A family wedding reignites the ancient feud be...   
3  False  31357  tt0114885  Cheated on, mistreated and stepped on, the wom...   
4  False  11862  tt0113041  Just when George Banks has recovered from his ...   

  release_date  runtime                                            tagline  \
0   1995-10-30     81.0                                                NaN   
1   1995-12-15    104.0          Roll the dice and unleash the excitement!   
2   1995-12-22    101.0  Still Yelling. Still Fighting. Still Ready for...   
3   1995-12-22    127.0  Friends are the people who let you be yourself...   
4   1995-02-10    106.0  Just When His World Is Back To Normal... He's ...   

                         title  vote_average

In [9]:
# procesamos los keywords del dataset keyword
import ast

keyword_column = keywords['keywords']

# Convertir las cadenas de texto que representan listas de diccionarios en listas de diccionarios reales
keyword_lists = keyword_column.apply(ast.literal_eval)

all_keyword_names = []
for keyword_list in keyword_lists:
    keyword_names = []
    for keyword_dict in keyword_list:
        keyword_names.append(keyword_dict['name'])
    all_keyword_names.append(keyword_names)

keywords['keywords'] = all_keyword_names
keywords.head()

,id,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."


In [10]:
# ahora vamos a añadir una columna keywords (de keywords)
# Convertir el tipo de datos de la columna 'id' en ambos DataFrames a objeto (str)
movies_new['id'] = movies_new['id'].astype(str)
keywords['id'] = keywords['id'].astype(str)

movies_new = movies_new.merge(keywords, on='id', how='left')

# Mostrar las primeras filas del DataFrame movies_new con la columna 'keyword' añadida
print(movies_new.head())

   adult     id    imdb_id                                           overview  \
0  False    862  tt0114709  Led by Woody, Andy's toys live happily in his ...   
1  False   8844  tt0113497  When siblings Judy and Peter discover an encha...   
2  False  15602  tt0113228  A family wedding reignites the ancient feud be...   
3  False  31357  tt0114885  Cheated on, mistreated and stepped on, the wom...   
4  False  11862  tt0113041  Just when George Banks has recovered from his ...   

  release_date  runtime                                            tagline  \
0   1995-10-30     81.0                                                NaN   
1   1995-12-15    104.0          Roll the dice and unleash the excitement!   
2   1995-12-22    101.0  Still Yelling. Still Fighting. Still Ready for...   
3   1995-12-22    127.0  Friends are the people who let you be yourself...   
4   1995-02-10    106.0  Just When His World Is Back To Normal... He's ...   

                         title  vote_average

In [12]:
# preprocesamos el cast para añadirlo al conjunto de datos final
import ast

credits_column = credits['cast']

# Convertir las cadenas de texto que representan listas de diccionarios en listas de diccionarios reales
credits_lists = credits_column.apply(ast.literal_eval)

all_credits_names = []
for credits_list in credits_lists:
    credits_names = []
    for credits_dict in credits_list:
        credits_names.append(credits_dict['name'])
    all_credits_names.append(credits_names)

credits.drop(columns='credits', inplace=True)
credits['cast'] = all_credits_names
credits.head()

,cast,crew,id
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[Whitney Houston, Angela Bassett, Loretta Devi...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [13]:
# ahora vamos a añadir una columna keywords (de keywords)
# Convertir el tipo de datos de la columna 'id' en ambos DataFrames a objeto (str)
movies_new['id'] = movies_new['id'].astype(str)
credits['id'] = credits['id'].astype(str)

movies_new = movies_new.merge(credits, on='id', how='left')

# Mostrar las primeras filas del DataFrame movies_new con la columna 'keyword' añadida
print(movies_new.head())

   adult     id    imdb_id                                           overview  \
0  False    862  tt0114709  Led by Woody, Andy's toys live happily in his ...   
1  False   8844  tt0113497  When siblings Judy and Peter discover an encha...   
2  False  15602  tt0113228  A family wedding reignites the ancient feud be...   
3  False  31357  tt0114885  Cheated on, mistreated and stepped on, the wom...   
4  False  11862  tt0113041  Just when George Banks has recovered from his ...   

  release_date  runtime                                            tagline  \
0   1995-10-30     81.0                                                NaN   
1   1995-12-15    104.0          Roll the dice and unleash the excitement!   
2   1995-12-22    101.0  Still Yelling. Still Fighting. Still Ready for...   
3   1995-12-22    127.0  Friends are the people who let you be yourself...   
4   1995-02-10    106.0  Just When His World Is Back To Normal... He's ...   

                         title  vote_average

In [14]:
# Se borran las columnas que ya no se necesitan
movies_new.drop(columns=['id', 'imdb_id', 'crew'], inplace=True)
print(movies_new.head())

   adult                                           overview release_date  \
0  False  Led by Woody, Andy's toys live happily in his ...   1995-10-30   
1  False  When siblings Judy and Peter discover an encha...   1995-12-15   
2  False  A family wedding reignites the ancient feud be...   1995-12-22   
3  False  Cheated on, mistreated and stepped on, the wom...   1995-12-22   
4  False  Just when George Banks has recovered from his ...   1995-02-10   

   runtime                                            tagline  \
0     81.0                                                NaN   
1    104.0          Roll the dice and unleash the excitement!   
2    101.0  Still Yelling. Still Fighting. Still Ready for...   
3    127.0  Friends are the people who let you be yourself...   
4    106.0  Just When His World Is Back To Normal... He's ...   

                         title  vote_average languages_availables  \
0                    Toy Story           7.7            [English]   
1             

In [16]:
# Defino un nuevo orden para ver los datos
new_order = ['title', 'genres_associated', 'languages_availables', 'runtime', 'overview', 'adult', 'release_date', 'tagline', 'vote_average']

# Reordenar las columnas del DataFrame movies_new
movies_new = movies_new.reindex(columns=new_order + [col for col in movies_new.columns if col not in new_order])

# Mostrar las primeras filas del DataFrame movies_new con las columnas reordenadas
print(movies_new.head())

                         title             genres_associated  \
0                    Toy Story   [Animation, Comedy, Family]   
1                      Jumanji  [Adventure, Fantasy, Family]   
2             Grumpier Old Men             [Romance, Comedy]   
3            Waiting to Exhale      [Comedy, Drama, Romance]   
4  Father of the Bride Part II                      [Comedy]   

  languages_availables  runtime  \
0            [English]     81.0   
1  [English, Français]    104.0   
2            [English]    101.0   
3            [English]    127.0   
4            [English]    106.0   

                                            overview  adult release_date  \
0  Led by Woody, Andy's toys live happily in his ...  False   1995-10-30   
1  When siblings Judy and Peter discover an encha...  False   1995-12-15   
2  A family wedding reignites the ancient feud be...  False   1995-12-22   
3  Cheated on, mistreated and stepped on, the wom...  False   1995-12-22   
4  Just when George Bank

In [18]:
# Especificar la ruta del archivo CSV de salida
output_csv_path = "/content/drive/MyDrive/dataIA/movies/movies_new.csv"

# Exportar el DataFrame movies_new a un archivo CSV
movies_new.to_csv(output_csv_path, index=False)

print(f"El archivo CSV se ha exportado exitosamente a {output_csv_path}")

El archivo CSV se ha exportado exitosamente a /content/drive/MyDrive/dataIA/movies/movies_new.csv


In [20]:
# Calcular el número de valores nulos en cada columna del DataFrame movies_new
null_counts = movies_new.isnull().sum()

# Mostrar el número de valores nulos en cada columna
print("Número de valores nulos en cada columna:")
print(null_counts)

Número de valores nulos en cada columna:
title                       0
genres_associated           0
languages_availables        0
runtime                   258
overview                  960
adult                       0
release_date               78
tagline                 25494
vote_average                0
keywords                    1
cast                        1
dtype: int64


In [22]:
# BORRO TODAS LAS FILAS QUE TIENEN ALGÜN NULL
# Eliminar todas las filas que contengan al menos un valor nulo en el DataFrame movies_new
movies_new_cleaned = movies_new.dropna()

null_counts = movies_new_cleaned.isnull().sum()

# Mostrar el número de valores nulos en cada columna
print("Número de valores nulos en cada columna:")
print(null_counts)

Número de valores nulos en cada columna:
title                   0
genres_associated       0
languages_availables    0
runtime                 0
overview                0
adult                   0
release_date            0
tagline                 0
vote_average            0
keywords                0
cast                    0
dtype: int64


In [24]:
# Especificar la ruta del archivo CSV de salida
output_csv_path = "/content/drive/MyDrive/dataIA/movies/movies_new_notna.csv"

# Exportar el DataFrame movies_new a un archivo CSV
movies_new_cleaned.to_csv(output_csv_path, index=False)

print(f"El archivo CSV se ha exportado exitosamente a {output_csv_path}")

El archivo CSV se ha exportado exitosamente a /content/drive/MyDrive/dataIA/movies/movies_new_notna.csv
